In [ ]:
import pandas as pd
import numpy as np
from tqdm import notebook
from tqdm import notebook

In [ ]:
# if the data is in form of a database
import pandas as pd
import sqlite3
import sqlalchemy 

try:
    conn = sqlite3.connect("lockdown1.db")    
    #Now in order to read in pandas dataframe we need to know table name
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM geo;")
    #print(f"Table Name : {cursor.fetchall()}")
    df = pd.read_sql_query('SELECT * FROM geo', conn)
    df.head()
    conn.close()

except Error as e:
    print(e)



In [ ]:
df = pd.read_csv('combined_files.txt',names=['tweet_id','sentiment_score']) 
# in combined_files.txt files I stored tweet_ids and sentiment scores of all the tweets
df.sort_values(['tweet_id'],inplace =True)
df.reset_index(drop=True,inplace = True)
df.head()

In [ ]:
import tweepy
#get your keys using twitter account
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)


In [ ]:
tweets = api.statuses_lookup([1248458710189363204], tweet_mode='extended') #get the entire tweet text
print(tweets[0].id) #displays tweet id
print(tweets[0].full_text) #displays the tweet text

In [ ]:
tweet_ids = list(df.tweet_id)
t_ids = [] #if tweet of a corresponding id is found then it's tweet_id will be stored here
t_texts = [] #if tweet of a corresponding id is found then it's tweet text will be stored here

In [ ]:
#first index where to start downloading tweets
# As it takes a lot of time to download tweets (as you can request only around 100 tweets per request)
#So if you had to stop training at some point just put first_idx to index in 'tweet_ids' list from where you want to continue
first_idx = 0
end_idx = first_idx

In [ ]:
flag = False
while end_idx <= df.shape[0]:
    if end_idx +100 >  df.shape[0]:
        end_idx = df.shape[0]
        flag=True
    else:
        end_idx += 100
        
    print(end_idx,end=",")
    tmp = api.statuses_lookup(tweet_ids[first_idx:end_idx], tweet_mode='extended')
    tmp_ids = []
    tmp_tweets = []
    for x in tmp:
        tmp_ids.append(x.id)
        tmp_tweets.append(x.full_text)
    t_texts += tmp_tweets
    t_ids += tmp_ids
    first_idx = end_idx
    if flag:
        break
    

In [ ]:
# to store tweet_ids and corresponding text if you wish to stop downloading
import pickle
with open('ids.pkl', 'wb') as f:
    pickle.dump(t_ids, f)
with open('texts.pkl', 'wb') as f:
    pickle.dump(t_texts, f)

In [ ]:
# to load previously downloaded tweets
with open('ids.pkl', 'rb') as f:
    t_ids1 = pickle.load(f)

    
with open('texts.pkl', 'rb') as f:
    t_texts1 = pickle.load(f)


In [ ]:
#if you had multiple checkpoints
all_t_ids = t_ids1+t_ids2+t_ids
all_t_texts = t_texts1+t_texts2+t_texts
len(all_t_ids),len(all_t_texts)

In [ ]:
dfx = pd.DataFrame({'tweet_id':all_t_ids,'text':all_t_texts})
dfx.head()

In [ ]:
# get sentiment scores of tweets of which text was found
df_p = 0
for i in notebook.tqdm(range(dfx.shape[0])):
    if dfx.tweet_id[i] == df.tweet_id[df_p]:
        sscores.append(df.sentiment_score[df_p])
        df_p+=1
    elif dfx.tweet_id[i] > df.tweet_id[i]:
        while True:
            df_p+=1
            if dfx.tweet_id[i] == df.tweet_id[df_p]:
                sscores.append(df.sentiment_score[df_p])
                df_p+=1
                break         

In [ ]:
dfx.sentiment_score = sscores #makes a new column

In [ ]:
dfx.to_csv('tweets_downloaded.csv',index =False)